In [1]:
import credentials
import requests
import pandas as pd
import tweepy as tw
import re
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
import string
import nltk
from snowballstemmer import stemmer
from collections import Counter
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /Users/deniz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/deniz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
try:
    auth = tw.OAuthHandler(credentials.API_Key, credentials.API_Key_Secret)
    auth.set_access_token(credentials.Access_Token, credentials.Access_Token_Secret)
    api = tw.API(auth)
    print("Successfully Authentication")
except:
    print("Failed Authentication")

Successfully Authentication


In [7]:
def user_info(username):
    user = api.get_user(screen_name=username)
    user_dict={
        'id': user.id,
        'name': user.name,
        'location': user.location,
        'description': user.description,
        'followers_count': user.followers_count,
        'following_count': user.friends_count,
        'create_date': (user.created_at).strftime('%d.%m.%y'),
        'favourites_count': user.favourites_count,
        'tweets_count: ': user.statuses_count,
        'liked_count': user.favourites_count
    }
    return user_dict
    
user_info("emin_adin")

In [6]:
# Access only the most recent 200 tweets with api.user_timeline. We access tweets that includes replied
# If want RT's include --> includerts=True
# If want with replies -->  exclude_replies=False. 

def get_user_tweets(username, limit, includerts=False, excludereply=True):
   tweets = api.user_timeline(screen_name=username, count=limit, include_rts=includerts, tweet_mode='extended', exclude_replies=excludereply)
   tweet_list = []
   columns = ['Date', 'Tweet']
   for tweet in tweets:
      tweet_list.append([(tweet.created_at).strftime('%d/%m/%y'), tweet.full_text])
   
   df = pd.DataFrame(tweet_list, columns=columns)
   print(df)

get_user_tweets('elonmusk', 20)

       Date                                              Tweet
0  02/04/23  Walked whole Cybertruck production line at Gig...
1  02/04/23  The real tragedy of @NYTimes is that their pro...
2  02/04/23                          ♥️♥️ @CommunityNotes ♥️♥️


In [ ]:
# You can access only the most recent 3200 tweets with tweepy.Cursor.
# If want RT's include --> includerts=True
# If want with replies -->  exclude_replies=False. 

def get_user_tweets_cursor1(username, limit, includerts=True, excludereply=False, tocsv=False):
   tweet_list = []
   columns = ['Date', 'Tweet', 'Likes', 'Retweet']
   tweets = tw.Cursor(api.user_timeline, screen_name=username, count=200, include_rts=includerts, exclude_replies=excludereply, tweet_mode='extended').items(limit)
   for tweet in tweets:
      tweet_list.append([(tweet.created_at).strftime('%d/%m/%y'), tweet.full_text, tweet.favorite_count, tweet.retweet_count]) 
   
   tweet_table = pd.DataFrame(tweet_list, columns=columns)

   if(tocsv):
      tweet_table.to_csv(f"{username}.csv", index=False)

   return tweet_table

get_user_tweets_cursor1('elonmusk', 20, False, True) # Only tweet that the user self-written and didn't include replies.

In [5]:
# You can access only the most recent 3000 tweets with tweepy.Cursor.

def get_user_tweets_cursor(username, limit, includerts=True, excludereply=False):
   tweets = tw.Cursor(api.user_timeline, screen_name=username, count=200, include_rts=includerts, exclude_replies=excludereply, tweet_mode='extended').items(limit)
   return tweets

In [7]:
def to_dataframe(tweets, columns=['created_at', 'full_text', 'retweet_count', 'favorite_count']):
    columns_naming = {
        'created_at': 'Creation Date',
        'id': 'ID',
        'id_str': 'ID as String',
        'full_text': 'Full Text',
        'truncated': 'Is Truncated',
        'display_text_range': 'Displayed Text Range',
        'entities': 'Entities',
        'extended_entities': 'Extended Entities',
        'source': 'Source',
        'in_reply_to_status_id': 'Replied to Status ID',
        'in_reply_to_status_id_str': 'Replied to Status ID as String',
        'in_reply_to_user_id': 'Replied to User ID',
        'in_reply_to_user_id_str': 'Replied to User ID as String',
        'in_reply_to_screen_name': 'Replied to Screen Name',
        'user': 'User',
        'geo': 'Geo Location',
        'coordinates': 'Coordinates',
        'place': 'Place',
        'contributors': 'Contributors',
        'is_quote_status': 'Is a Quote Status',
        'retweet_count': 'Retweet Count',
        'favorite_count': 'Favorite Count',
        'favorited': 'Is Favorited',
        'retweeted': 'Is Retweeted',
        'possibly_sensitive': 'Possibly Sensitive',
        'lang': 'Language'
    }

    tweet_list = []
    for tweet in tweets:
        tweet_dict = tweet._json
        filter_tweets = [tweet_dict[cn] for cn in columns]
        tweet_list.append(filter_tweets)


    columns_name = [columns_naming[key] for key in columns]
    tweets_table = pd.DataFrame(tweet_list, columns=columns_name)
    if 'Creation Date' in tweets_table.columns:
        tweets_table['Creation Date'] = pd.to_datetime(tweets_table['Creation Date']).dt.strftime('%d/%m/%y')
    return tweets_table

In [8]:
def to_csv(tweets_table, username="tweets"):
    tweets_table.to_csv(f"{username}.csv", index=False)

In [165]:
def filter_tweets(tweets):
    en_useless_words = list(string.punctuation) + nltk.corpus.stopwords.words('english')
    tr_useless_words = list(string.punctuation) + nltk.corpus.stopwords.words('turkish')
    tr_filtered_text = []
    en_filtered_text = []
    raw_text_list = []
    for tweet in tweets:
        text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet._json['full_text'])
        tweet_words = nltk.word_tokenize(text)
        raw_text_list.extend(tweet_words)
        if tweet.lang == 'tr':
            tr_filtered_text.extend([word for word in tweet_words if not (word in tr_useless_words or re.match('^[^\w]*$', word))])
        if tweet.lang == 'en':
            en_filtered_text.extend([word for word in tweet_words if not (word in en_useless_words or re.match('^[^\w]*$', word))])
    return en_filtered_text, tr_filtered_text, raw_text_list

In [166]:
def most_common_words(en_words, tr_words, count):
    all_words = en_words + tr_words
    words_count = Counter(all_words)
    most_common = words_count.most_common()[:count]
    
    df = pd.DataFrame(most_common, columns=['Word', 'Count'])
    return df

In [167]:
def visualize_common_words(en_words, tr_words, count):
    all_words = en_words + tr_words
    words_count = Counter(all_words)
    most_common = words_count.most_common()[:count]
    df = pd.DataFrame(most_common, columns=['Word', 'Count'])

    plt.bar(df['Word'], df['Count'], color=(0.2, 0.4, 0.6, 0.6))
    plt.title('Word Frequency')
    plt.xlabel('Word')
    plt.ylabel('Count')
    plt.xticks(rotation=90)
    plt.show()

In [188]:
def most_fav_tweet(tweets):
    most_fav_tweet = None
    max_fav = -1
    for tweet in tweets:
        if tweet.favorite_count > max_fav:
            max_fav = tweet.favorite_count
            most_fav_tweet = tweet.full_text
    return (most_fav_tweet, max_fav)

In [190]:
def all_fonk():
    
    all_tweets = get_user_tweets_cursor('emin_adin', 50, False, True) # Only tweet that the user self-written and didn't include replies.
    most_fav_tweet(all_tweets)

all_fonk()

In [3]:
def get_all_tweets(screen_name, max_tweets=500, includerts=True, excludereply=False):
    last_id = None

    all_tweets = []
    while len(all_tweets) < max_tweets:
        new_tweets = api.user_timeline(screen_name=screen_name, count=200, max_id=last_id, include_rts=includerts, exclude_replies=excludereply)
        if not new_tweets:
            break
        all_tweets.extend(new_tweets)
        last_id = new_tweets[-1].id -1
    return all_tweets

In [ ]:
client = tw.Client(bearer_token=credentials.Bearer_Token)

In [19]:
# We can use Client for get User's Tweet

def get_user_tweets_client(username):

    user = client.get_user(username=username).data
    userid=user.id

    tweets =client.get_users_tweets(id=userid, tweet_fields=['id', 'text', 'created_at', 'context_annotations'])
    
    for tweet in tweets.data:
        print((tweet.created_at).strftime('%d/%m/%y'), '\n', tweet.text, '\n', tweet.context_annotations, '\n')

get_user_tweets_client('emin_adin')

2023-03-29 16:10:51+00:00 
 Şeker ve yüksek şekerli içecekler, hele de fruktoz şurubu içerenler, dostunuz değildir. https://t.co/NR3jYFeLv5 
 [{'domain': {'id': '46', 'name': 'Business Taxonomy', 'description': 'Categories within Brand Verticals that narrow down the scope of Brands'}, 'entity': {'id': '1557696888568045569', 'name': 'Food & Beverage Business', 'description': 'Brands, companies, advertisers and every non-person handle with the profit intent and related to foods & beverages such as snacks, soft drinks, beers.'}}] 

2023-03-28 16:17:17+00:00 
 iPhone kutularını boşuna saklamayın, taşınınca çöpe gidiyor :) ♻️ https://t.co/fGuN7cdVhv 
 [{'domain': {'id': '46', 'name': 'Business Taxonomy', 'description': 'Categories within Brand Verticals that narrow down the scope of Brands'}, 'entity': {'id': '1557697333571112960', 'name': 'Technology Business', 'description': 'Brands, companies, advertisers and every non-person handle with the profit intent related to softwares, apps, comm

In [9]:
"""
    Twitter API Without Tweepy
    Authorization and Search 
"""

def create_bearer():
    url = "https://api.twitter.com/oauth2/token"
    auth = (credentials.API_Key, credentials.API_Key_Secret)
    data = {'grant_type': 'client_credentials'}
    response = requests.post(url, data=data, auth=auth)
    return response.json()

bearer_token = create_bearer()

def get_followers_list(screen_name):
    url = f"https://api.twitter.com/1.1/followers/list.json?cursor=-1&screen_name={screen_name}&skip_status=true&include_user_entities=false"
    headers = {'Authorization': f"{bearer_token['token_type']} {bearer_token['access_token']}"}
    response = requests.get(url, headers=headers)
    return response.json()

def search_tweet(query):
        
    url = f"https://api.twitter.com/1.1/search/tweets.json?q={query}&result_type=popular&count=10"
    headers = {'Authorization': f"{bearer_token['token_type']} {bearer_token['access_token']}"}
    response = requests.get(url, headers=headers)
    return response.json()    

#get_followers_list('coingirl13')
#tweets = search_tweet('nashville')
